In [1]:
import json
train = []
with open('train') as f:
    for line in f:
        train.append(json.loads(line.strip()))
val = []
with open('val') as f:
    for line in f:
        val.append(json.loads(line.strip()))
with open('test_no_reference') as f:
  test = f.readlines()

In [2]:
print(train[:3])

[{'dst': '- Intriguing.', 'src': '◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵'}, {'dst': 'He would need to repeat his vows in the land of the living and drink from the wine of ages.', 'src': '▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦ ◫◉◎▴▱◫▵'}, {'dst': "You couldn't even answer my texts?", 'src': '◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?'}]


In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import sys
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.optim import Adam
import os
import subprocess

from matplotlib import pyplot as plt
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import shutil
import random
import itertools
from datasets import load_metric

bleu_metric = load_metric("bleu")

C:\Users\Katie\AppData\Local\Temp\ipykernel_16988\1372133618.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")
c:\Users\Katie\miniconda3\envs\mltest\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [4]:
dst=[]
src=[]
for i in range(len(train)):
    dst.append(train[i]['dst'])
    src.append(train[i]['src'])


In [5]:
src

['◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵',
 '▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦ ◫◉◎▴▱◫▵',
 '◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?',
 '▯◪ ▨◠◈◠◓ ◞◭◓◠▫ ◳◠▻◬◳◧◓ ◞▴▦◗▦▨◫?',
 '◈◠ ◧▱◠▦ ◀◫◓ ▨◠◉ ◂▱◠▽◈◠▦ ◀◠▷◞◪◈◗◳◧◓■ ◉◧◐▾▦▱◨◐▾ ○▱◎◠▦▱◠◓◈◠▦ ▨◠◉◠▦ ▽◠▷◨◈◫▱▴◓▵',
 '◝▴▦ ◫◒▫◪▦ ▨◠◳▫◠◓◠▼◠▨ ◀◗◓◫ ◈▴◐◫▱◈◗◓▵',
 '▭◂◒◉◠ ▨◠▱ ◡◠▨◂◀▵',
 '◝◨◓◠◈◠▨◗ ▷◪◓▨◪◞◫ ◳◠▽◠ ◀◬◓◠▨▪◓▵',
 '◝◪▨▱▴ ◀◗◓ ◞◠▦◗▽▴!',
 '◁◠◎◠▦ ◒◗◎◈◫▵',
 '◢◓▴◈◗ ◚◪◓▴▦▱◪◓◫◎◫▢ ◀◂◓◉▱◠◓▪◎▪▢◬ ◞◠▫▪▦ ◠▱◠◓◠▨ ▦◂▫ ◈◪◍▫◪◓▱◪◓◫◳▱◪ ◧▽ ◚▴◓◫◳◂◓▱◠◓▵',
 '◤▴◓◗▦◫▢■ ▷◪◎▴▦ ◒▾◓◠◞▪▵',
 '◱▵▵▵◎◧◓ ◉◨▱▱◨▨ ◧▱◨◓◈◨▵',
 '◝◫◓ ◠◓▨◠◈◠◒▪◎▱◠ ◀◨▱◨◒◠▼◠◐▪◎▵',
 '◊▨◫ ◞◂▽▱◨ ◠◗▱▴ ◠◓◠◞▪▦◈◠▨◗ ◀◠◓◬◒▪ ◞◠◐▱◠◎◠▨▵',
 '□◠▫◓◂▦◨◎ ◀◠▦◠ ▫▴▨◎▴ ◚◪▽◠ ▫◂▨◠▫ ◠▫◞◠ ◀◗▱◪ ◉◠▱◬◒▫▪◎▵',
 '▮▴▦◫ ◀◗▱▴ ◳▴◓◗◎▵ ▮◭▻▴◓ ◄◠◓◗◂▵',
 '○◎◠ 12 ▨◗◒◗▱▴◓▵ ► ◙▴◓▫ ◪▫◎◪▵',
 "○▱▫◬▦ ◖◪▷◫◓'◗▦ ▨◓◠▱◗◉▴◞◫",
 '◝▴▦◗◎ ◫◒▫▴▦ ▨◠◉▫▪◐◬◎▪ ◕◇◓◎◭▽◧◓◞▾▦▵',
 '◝◪▦ ◞◠◈▴▼▴ ◗◳◗ ◀◗◓ ◈◫▢◫ ◉▴▨▫◫◐◗◎◗▢◫ ◞◇◳▱◭▽◧◓▾◎▵',
 '▢◠▫◪▦ ◀▴◈◪▱◫▦◫ ◇◈▩◳◧◓◨◎; ▻▴▨◗ ◧▦◠ ▦◪ ▽◠▻◠▼◠▨◞▪▦?',
 '◝▾ ◫◒◫ ◀◫◓ ◒◪▨◫▱◈◪▵▵▵ ► ▭◗◉ ◒◠▦◞◬▦ ◳◧▨▵',
 '◝▾ ▫◠▫▫▪◐◬▦ ▴▦ ◫◳◫ ◀◠◓◀◪▨◭ ▶◓◠◚ ◕◠◓◠▦▫◫ ▴◈◪◓◗◎▵',
 '◆◠◳▴▫ ◗▽◗ ◕◣◓▩▦▩▽◂◓

In [6]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [7]:
tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

In [8]:
tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

In [9]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=100000, min_frequency=2, special_tokens=special_tokens)

In [10]:
def get_training_corpus():
    for i in range(0, len(src)):
        yield src[i]

In [11]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [27]:
tokenizer.model = models.WordPiece(unk_token="[UNK]")

tokenizer.train( src, trainer=trainer)

Exception: The system cannot find the file specified. (os error 2)

In [41]:
new_tokenizer = Tokenizer.from_file("tokenizer.json")
new_tokenizer.model = models.WordPiece(unk_token="[UNK]")

In [30]:
encoding = new_tokenizer.encode("Let's test this tokenizer.")
print(encoding.ids)

[2, 0, 0, 0, 0, 3]


In [19]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [31]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [33]:
encoding = new_tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

['[CLS]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[SEP]']


In [22]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [34]:
new_tokenizer.decode([1000, 2000, 3000, 400, 5000, 6000])

'◗◞▫▴ ▻▴▨◭◒▫▼▴ ◂▱◠◀◗▱◗◓ ◢◓'

In [ ]:
tokenizer.save("tokenizer.json")

In [42]:
new_tokenizer.encode("► ◊◒▴ ▽◠◓◠◳◠▼◠▨■ ▱◂▱◗▻◧▻ ◕◫◀◗▵").ids

Exception: WordPiece error: Missing [UNK] token from the vocabulary

In [36]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    # tokenizer_file="tokenizer.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

c:\Users\Katie\miniconda3\envs\mltest\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [37]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [43]:
wrapped_tokenizer.tokenize("► ◒ ▽◳◠ ▱◂▱◗▻◧▻ ◕◫◗")

Exception: WordPiece error: Missing [UNK] token from the vocabulary

In [ ]:
import spacy
en_nlp = spacy.load("en_core_web_sm")


In [ ]:
from datasets import load_dataset
train_zet = load_dataset('json', data_files='train', split="train")
valid_zet = load_dataset('json', data_files='val' , split="train")


In [ ]:
train = train_zet
valid = valid_zet

In [ ]:
train[:10]

{'dst': ['- Intriguing.',
  'He would need to repeat his vows in the land of the living and drink from the wine of ages.',
  "You couldn't even answer my texts?",
  'How fast do you go?',
  "He's talking about a few right here in Lisbon, mostly Jews who have escaped the Germans, that's all.",
  'For instance, Ben is not a guy who skips work.',
  'Goodbye, Jakob.',
  'Sure beats the foam domes round here.',
  'Oh, wait a second!',
  'The time is now.'],
 'src': ['◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵',
  '▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦ ◫◉◎▴▱◫▵',
  '◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?',
  '▯◪ ▨◠◈◠◓ ◞◭◓◠▫ ◳◠▻◬◳◧◓ ◞▴▦◗▦▨◫?',
  '◈◠ ◧▱◠▦ ◀◫◓ ▨◠◉ ◂▱◠▽◈◠▦ ◀◠▷◞◪◈◗◳◧◓■ ◉◧◐▾▦▱◨◐▾ ○▱◎◠▦▱◠◓◈◠▦ ▨◠◉◠▦ ▽◠▷◨◈◫▱▴◓▵',
  '◝▴▦ ◫◒▫◪▦ ▨◠◳▫◠◓◠▼◠▨ ◀◗◓◫ ◈▴◐◫▱◈◗◓▵',
  '▭◂◒◉◠ ▨◠▱ ◡◠▨◂◀▵',
  '◝◨◓◠◈◠▨◗ ▷◪◓▨◪◞◫ ◳◠▽◠ ◀◬◓◠▨▪◓▵',
  '◝◪▨▱▴ ◀◗◓ ◞◠▦◗▽▴!',
  '◁◠◎◠▦ ◒◗◎◈◫▵']}